# Demo the Text-to-Speech module

In [ ]:
import glados.tts as tts
import sounddevice as sd

In [ ]:
glados_tts = tts.Synthesizer(
            model_path=str("models/glados.onnx"),
            use_cuda=True,
            speaker_id=0,
        )

In [ ]:
# Generate the audio.
audio = glados_tts.generate_speech_audio("Hello, my name is Glados. I am an AI created by Aperture Science.")

# Play the audio
sd.play(audio, glados_tts.rate)

# Demo the Automatic speech recogntion system
This will detect and transcribe your voice. In this demo, it will then get GlaDOS to repeat back to you what was heard.

In [ ]:
import glados.voice_recognition as vr

In [ ]:
def say_text(text: str):
    """Say text using text-to-speech engine
    """
    audio = glados_tts.generate_speech_audio(text)
    sd.play(audio, tts.RATE)
    sd.wait()

# Instantiate VoiceRecognition class with the say_text function
demo = vr.VoiceRecognition(function=say_text)

# Start the demo
demo.start()